In [2]:
import pandas as pd
from run_trial import *
import pickle

/home/yongweic/.conda/envs/contrastive2/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/yongweic/.conda/envs/contrastive2/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/yongweic/.conda/envs/contrastive2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-06 13:42:48.554367: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot 

In [3]:
# Path to CSV generated by experiments
CSV_PATH = ''
df = pd.read_csv(CSV_PATH)

In [ ]:
l2_penalty = 0.00

path = df.loc[(df['trial']==13) 
              & (df['feature_dim'] == 8) 
              & (df['temperature'] == 0.01)
              & (df['l2_penalty'] ==l2_penalty)
              , ['experimentInfo_path']].reset_index(drop=True)['experimentInfo_path'][0]
print(path)


# Define line types for different model types
LSE_linestyle = '--'
direct_linestyle = '-'

# Define colors for LSE and direct models
colors = ['grey', 'red', 'blue', 'green', 'black']


# Open and read the pickle file
with open(path, 'rb') as file:
    experimentInfo = pickle.load(file)

# Now, experimentInfo contains the object that was pickled at the specified path
# You can access its attributes or data as needed, for example:
distribution, models, accuracies = experimentInfo

for model_type, acc in accuracies.items():
    if 'norm' in model_type:
        model_idx = 0
    elif 'dist' in model_type:
        model_idx = 1
    elif 'dot' in model_type:
        model_idx = 2
    elif 'unconnected' in model_type:
        model_idx = 3
    else:
        model_idx = 4

    color = colors[model_idx]

    if 'LSE' in model_type:
        linestyle = LSE_linestyle
    elif 'Direct' in model_type:
        linestyle = direct_linestyle

    if model_type == 'conscious_model':
        plt.plot(acc,label='conscious', color=color)
    else:
        plt.plot([x[0] for x in acc], label=(f'{model_type} (Direct)'), color=color, linestyle='-')
        plt.plot([x[1] for x in acc], label=(f'{model_type} LSE'), color=color, linestyle='--')
    
plt.title(f'Validation Accuracy of InfoNCE Models')
plt.legend()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd

# Assuming df is defined and contains the necessary columns
# l2_penalty, path, etc. are defined as given

l2_penalty = 0.00

# This path assignment is based on the original code provided
path = df.loc[(df['trial']==13) 
              & (df['feature_dim'] == 8) 
              & (df['temperature'] == 0.01)
              & (df['l2_penalty'] == l2_penalty)
              , ['experimentInfo_path']].reset_index(drop=True)['experimentInfo_path'][0]
print(path)

# Define line types for different model types
LSE_linestyle = '--'
direct_linestyle = '-'

# Define colors for LSE and direct models
colors = ['grey', 'red', 'blue', 'green', 'black']

# Open and read the pickle file
with open(path, 'rb') as file:
    experimentInfo = pickle.load(file)

# Extract data from experimentInfo
distribution, models, accuracies = experimentInfo

# Initialize a dictionary to hold accuracies for each model type across trials
all_accuracies = {model_type: [] for model_type in accuracies.keys()}

# Simulate the data structure by appending each trial's accuracies to the respective list
for trial in range(20):  # Assuming 20 trials labelled 0-19
    for model_type, acc in accuracies.items():
        all_accuracies[model_type].append(acc)

# Calculate mean and std deviation for each model type
mean_accuracies = {}
std_accuracies = {}

for model_type, acc_list in all_accuracies.items():
    acc_array = np.array(acc_list)
    mean_accuracies[model_type] = np.mean(acc_array, axis=0)
    std_accuracies[model_type] = np.std(acc_array, axis=0)

# Plotting the mean accuracies with error bars
for model_type, mean_acc in mean_accuracies.items():
    std_acc = std_accuracies[model_type]
    
    if 'norm' in model_type:
        model_idx = 0
    elif 'dist' in model_type:
        model_idx = 1
    elif 'dot' in model_type:
        model_idx = 2
    elif 'unconnected' in model_type:
        model_idx = 3
    else:
        model_idx = 4

    color = colors[model_idx]

    if 'LSE' in model_type:
        linestyle = LSE_linestyle
    elif 'Direct' in model_type:
        linestyle = direct_linestyle

    if model_type == 'conscious_model':
        plt.errorbar(range(len(mean_acc)), mean_acc, yerr=std_acc, label='conscious', color=color, linestyle=direct_linestyle)
    else:
        plt.errorbar(range(len(mean_acc)), mean_acc[:, 0], yerr=std_acc[:, 0], label=(f'{model_type} (Direct)'), color=color, linestyle='-')
        plt.errorbar(range(len(mean_acc)), mean_acc[:, 1], yerr=std_acc[:, 1], label=(f'{model_type} LSE'), color=color, linestyle='--')

plt.title('Validation Accuracy of InfoNCE Models')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd

# Assuming df is already defined and contains the necessary columns
# l2_penalty, feature_dim, temperature, etc. are defined as given

l2_penalty = 0.00

# Collect all paths for the trials
paths = df.loc[(df['feature_dim'] == 8) 
               & (df['temperature'] == 0.01) 
               & (df['l2_penalty'] == l2_penalty), 
               ['trial', 'experimentInfo_path']].sort_values('trial')['experimentInfo_path']

# Define line types for different model types
LSE_linestyle = '--'
direct_linestyle = '-'

# Define colors for LSE and direct models
colors = ['grey', 'red', 'blue', 'green', 'black']

# Initialize a dictionary to hold accuracies for each model type across trials
all_accuracies = {}

# Loop over each path and extract accuracies
for path in paths:
    with open(path, 'rb') as file:
        experimentInfo = pickle.load(file)
    # Extract data from experimentInfo
    _, _, accuracies = experimentInfo
    # Append accuracies to all_accuracies dictionary
    for model_type, acc in accuracies.items():
        if model_type not in all_accuracies:
            all_accuracies[model_type] = []
        all_accuracies[model_type].append(acc)

# Calculate mean and std deviation for each model type
mean_accuracies = {}
std_accuracies = {}

for model_type, acc_list in all_accuracies.items():
    acc_array = np.array(acc_list)
    mean_accuracies[model_type] = np.mean(acc_array, axis=0)
    std_accuracies[model_type] = np.std(acc_array, axis=0)

# Plotting the mean accuracies with error bars
for model_type, mean_acc in mean_accuracies.items():
    std_acc = std_accuracies[model_type]
    
    if 'norm' in model_type:
        model_idx = 0
    elif 'dist' in model_type:
        model_idx = 1
    elif 'dot' in model_type:
        model_idx = 2
    elif 'unconnected' in model_type:
        model_idx = 3
    else:
        model_idx = 4

    color = colors[model_idx]

    if 'LSE' in model_type:
        linestyle = LSE_linestyle
    elif 'Direct' in model_type:
        linestyle = direct_linestyle

    if model_type == 'conscious_model':
        plt.errorbar(range(len(mean_acc)), mean_acc, yerr=std_acc, label='conscious', color=color, linestyle=direct_linestyle)
    else:
        plt.errorbar(range(len(mean_acc)), mean_acc[:, 0], yerr=std_acc[:, 0], label=(f'{model_type} (Direct)'), color=color, linestyle='-')
        plt.errorbar(range(len(mean_acc)), mean_acc[:, 1], yerr=std_acc[:, 1], label=(f'{model_type} LSE'), color=color, linestyle='--')

plt.title('Validation Accuracy of InfoNCE Models')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd

# Assuming df is already defined and contains the necessary columns
# l2_penalty, feature_dim, temperature, etc. are defined as given

l2_penalty = 0.00

# Collect all paths for the trials
paths = df.loc[(df['feature_dim'] == 8) 
               & (df['temperature'] == 0.01) 
               & (df['l2_penalty'] == l2_penalty), 
               ['trial', 'experimentInfo_path']].sort_values('trial')['experimentInfo_path']

# Define line types for different model types
LSE_linestyle = '--'
direct_linestyle = '-'

# Define colors for LSE and direct models
colors = ['grey', 'red', 'blue', 'green', 'black']

# Initialize a dictionary to hold accuracies for each model type across trials
all_accuracies = {}

# Loop over each path and extract accuracies
for path in paths:
    with open(path, 'rb') as file:
        experimentInfo = pickle.load(file)
    # Extract data from experimentInfo
    _, _, accuracies = experimentInfo
    # Append accuracies to all_accuracies dictionary
    for model_type, acc in accuracies.items():
        if model_type not in all_accuracies:
            all_accuracies[model_type] = []
        all_accuracies[model_type].append(acc)

# Calculate mean and std deviation for each model type
mean_accuracies = {}
std_accuracies = {}

for model_type, acc_list in all_accuracies.items():
    acc_array = np.array(acc_list)
    mean_accuracies[model_type] = np.mean(acc_array, axis=0)
    std_accuracies[model_type] = np.std(acc_array, axis=0)

# Plotting the mean accuracies with error bars
for model_type, mean_acc in mean_accuracies.items():
    std_acc = std_accuracies[model_type]
    
    if 'norm' in model_type:
        model_idx = 0
    elif 'dist' in model_type:
        model_idx = 1
    elif 'dot' in model_type:
        model_idx = 2
    elif 'unconnected' in model_type:
        model_idx = 3
    else:
        model_idx = 4

    color = colors[model_idx]

    if 'LSE' in model_type:
        linestyle = LSE_linestyle
    elif 'Direct' in model_type:
        linestyle = direct_linestyle

    if model_type == 'conscious_model':
        plt.errorbar(range(len(mean_acc)), mean_acc, yerr=std_acc, label='Ground Truth', color=color, linestyle=direct_linestyle)
    elif model_type == 'unconnected_model':
        plt.errorbar(range(len(mean_acc)), mean_acc[:, 0], yerr=std_acc[:, 0], label=(f'Unconnected (Direct)'), color='green', linestyle='-')
        plt.errorbar(range(len(mean_acc)), mean_acc[:, 1], yerr=std_acc[:, 1], label=(f'Unconnected (Monte Carlo)'), color='darkgreen', linestyle='--')

plt.title('Validation Accuracy of Unconnected Model')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd

# Assuming df is already defined and contains the necessary columns
# l2_penalty, feature_dim, temperature, etc. are defined as given

l2_penalty = 0.00

# Collect all paths for the trials
paths = df.loc[(df['feature_dim'] == 16) 
               & (df['temperature'] == 0.01) 
               & (df['l2_penalty'] == l2_penalty), 
               ['trial', 'experimentInfo_path']].sort_values('trial')['experimentInfo_path']

# Define line types for different model types
LSE_linestyle = '--'
direct_linestyle = '-'

# Define colors for LSE and direct models
colors = ['grey', 'red', 'blue', 'green', 'black']

# Initialize a dictionary to hold accuracies for each model type across trials
all_accuracies = {}

# Loop over each path and extract accuracies
for path in paths:
    with open(path, 'rb') as file:
        experimentInfo = pickle.load(file)
    # Extract data from experimentInfo
    _, _, accuracies = experimentInfo
    # Append accuracies to all_accuracies dictionary
    for model_type, acc in accuracies.items():
        if model_type not in all_accuracies:
            all_accuracies[model_type] = []
        all_accuracies[model_type].append(acc)

# Calculate mean and std deviation for each model type
mean_accuracies = {}
std_accuracies = {}

for model_type, acc_list in all_accuracies.items():
    acc_array = np.array(acc_list)
    mean_accuracies[model_type] = np.mean(acc_array, axis=0)
    std_accuracies[model_type] = np.std(acc_array, axis=0)

# Plotting the mean accuracies with error bars
for model_type, mean_acc in mean_accuracies.items():
    std_acc = std_accuracies[model_type]
    
    if 'norm' in model_type:
        model_idx = 0
    elif 'dist' in model_type:
        model_idx = 1
    elif 'dot' in model_type:
        model_idx = 2
    elif 'unconnected' in model_type:
        model_idx = 3
    else:
        model_idx = 4

    color = colors[model_idx]

    if 'LSE' in model_type:
        linestyle = LSE_linestyle
    elif 'Direct' in model_type:
        linestyle = direct_linestyle

    if model_type == 'conscious_model':
        plt.errorbar(range(len(mean_acc)), mean_acc, yerr=std_acc, label='Ground Truth', color=color, linestyle=direct_linestyle)
    elif model_type == 'dist_model':
        plt.errorbar(range(len(mean_acc)), mean_acc[:, 0], yerr=std_acc[:, 0], label=(f'L2 Dist (Direct)'), color=color, linestyle='-')
        plt.errorbar(range(len(mean_acc)), mean_acc[:, 1], yerr=std_acc[:, 1], label=(f'L2 Dist (Monte Carlo)'), color='orange', linestyle='--')
#     elif model_type == 'norm_model':
#         plt.errorbar(range(len(mean_acc)), mean_acc[:, 0], yerr=std_acc[:, 0], label=(f'Normed Dot Product (Direct)'), color='purple', linestyle='-')
#         plt.errorbar(range(len(mean_acc)), mean_acc[:, 1], yerr=std_acc[:, 1], label=(f'Normed Dot Product (Monte Carlo)'), color='indigo', linestyle='--')

plt.title('Validation Accuracy of L2 Dist Model')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd

# Assuming df is already defined and contains the necessary columns
# l2_penalty, feature_dim, temperature, etc. are defined as given

l2_penalty = 0.00

# Collect all paths for the trials
paths = df.loc[(df['feature_dim'] == 8) 
               & (df['temperature'] == 0.01) 
               & (df['l2_penalty'] == l2_penalty), 
               ['trial', 'experimentInfo_path']].sort_values('trial')['experimentInfo_path']

# Define line types for different model types
LSE_linestyle = '--'
direct_linestyle = '-'

# Define colors for LSE and direct models
colors = ['grey', 'red', 'blue', 'green', 'black']

# Initialize a dictionary to hold accuracies for each model type across trials
all_accuracies = {}

# Loop over each path and extract accuracies
for path in paths:
    with open(path, 'rb') as file:
        experimentInfo = pickle.load(file)
    # Extract data from experimentInfo
    _, _, accuracies = experimentInfo
    # Append accuracies to all_accuracies dictionary
    for model_type, acc in accuracies.items():
        if model_type not in all_accuracies:
            all_accuracies[model_type] = []
        all_accuracies[model_type].append(acc)

# Calculate mean and std deviation for each model type
mean_accuracies = {}
std_accuracies = {}

for model_type, acc_list in all_accuracies.items():
    acc_array = np.array(acc_list)
    mean_accuracies[model_type] = np.mean(acc_array, axis=0)
    std_accuracies[model_type] = np.std(acc_array, axis=0)

# Plotting the mean accuracies with error bars
for model_type, mean_acc in mean_accuracies.items():
    std_acc = std_accuracies[model_type]
    
    if 'norm' in model_type:
        model_idx = 0
    elif 'dist' in model_type:
        model_idx = 1
    elif 'dot' in model_type:
        model_idx = 2
    elif 'unconnected' in model_type:
        model_idx = 3
    else:
        model_idx = 4

    color = colors[model_idx]

    if 'LSE' in model_type:
        linestyle = LSE_linestyle
    elif 'Direct' in model_type:
        linestyle = direct_linestyle

    if model_type == 'conscious_model':
        plt.errorbar(range(len(mean_acc)), mean_acc, yerr=std_acc, label='Ground Truth', color=color, linestyle=direct_linestyle)
#     elif model_type == 'dist_model':
#         plt.errorbar(range(len(mean_acc)), mean_acc[:, 0], yerr=std_acc[:, 0], label=(f'L2 Dist (Direct)'), color=color, linestyle='-')
#         plt.errorbar(range(len(mean_acc)), mean_acc[:, 1], yerr=std_acc[:, 1], label=(f'L2 Dist (Monte Carlo)'), color='orange', linestyle='--')
    elif model_type == 'norm_model':
        plt.errorbar(range(len(mean_acc)), mean_acc[:, 0], yerr=std_acc[:, 0], label=(f'Normed Dot Product (Direct)'), color='purple', linestyle='-')
        plt.errorbar(range(len(mean_acc)), mean_acc[:, 1], yerr=std_acc[:, 1], label=(f'Normed Dot Product (Monte Carlo)'), color='indigo', linestyle='--')

plt.title('Validation Accuracy of Normed Dot Model')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd

# Assuming df is already defined and contains the necessary columns
# l2_penalty, feature_dim, temperature, etc. are defined as given

l2_penalty = 0.00

# Collect all paths for the trials
paths = df.loc[(df['feature_dim'] == 8) 
               & (df['temperature'] == 0.01) 
               & (df['l2_penalty'] == l2_penalty), 
               ['trial', 'experimentInfo_path']].sort_values('trial')['experimentInfo_path']

# Define line types for different model types
LSE_linestyle = '--'
direct_linestyle = '-'

# Define colors for LSE and direct models
colors = ['grey', 'red', 'blue', 'green', 'black']

# Initialize a dictionary to hold accuracies for each model type across trials
all_accuracies = {}

# Loop over each path and extract accuracies
for path in paths:
    with open(path, 'rb') as file:
        experimentInfo = pickle.load(file)
    # Extract data from experimentInfo
    _, _, accuracies = experimentInfo
    # Append accuracies to all_accuracies dictionary
    for model_type, acc in accuracies.items():
        if model_type not in all_accuracies:
            all_accuracies[model_type] = []
        all_accuracies[model_type].append(acc)

# Calculate mean and std deviation for each model type
mean_accuracies = {}
std_accuracies = {}

for model_type, acc_list in all_accuracies.items():
    acc_array = np.array(acc_list)
    mean_accuracies[model_type] = np.mean(acc_array, axis=0)
    std_accuracies[model_type] = np.std(acc_array, axis=0)

# Plotting the mean accuracies with error bars
for model_type, mean_acc in mean_accuracies.items():
    std_acc = std_accuracies[model_type]
    
    if 'norm' in model_type:
        model_idx = 0
    elif 'dist' in model_type:
        model_idx = 1
    elif 'dot' in model_type:
        model_idx = 2
    elif 'unconnected' in model_type:
        model_idx = 3
    else:
        model_idx = 4

    color = colors[model_idx]

    if 'LSE' in model_type:
        linestyle = LSE_linestyle
    elif 'Direct' in model_type:
        linestyle = direct_linestyle

    if model_type == 'conscious_model':
        plt.errorbar(range(len(mean_acc)), mean_acc, yerr=std_acc, label='Ground Truth', color=color, linestyle=direct_linestyle)
    elif model_type == 'dot_model':
        plt.errorbar(range(len(mean_acc)), mean_acc[:, 0], yerr=std_acc[:, 0], label=(f'Dot Product (Direct)'), color=color, linestyle='-')
        plt.errorbar(range(len(mean_acc)), mean_acc[:, 1], yerr=std_acc[:, 1], label=(f'Dot Product (Monte Carlo)'), color=color, linestyle='--')
#     elif model_type == 'norm_model':
#         plt.errorbar(range(len(mean_acc)), mean_acc[:, 0], yerr=std_acc[:, 0], label=(f'Normed Dot Product (Direct)'), color='purple', linestyle='-')
#         plt.errorbar(range(len(mean_acc)), mean_acc[:, 1], yerr=std_acc[:, 1], label=(f'Normed Dot Product (Monte Carlo)'), color='indigo', linestyle='--')

plt.title('Validation Accuracy of Dot Model')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd

# Assuming df is already defined and contains the necessary columns
# l2_penalty, feature_dim, temperature, etc. are defined as given

l2_penalty = 0.00

# Collect all paths for the trials
paths = df.loc[(df['feature_dim'] == 8) 
               & (df['temperature'] == 0.01) 
               & (df['l2_penalty'] == l2_penalty), 
               ['trial', 'experimentInfo_path']].sort_values('trial')['experimentInfo_path']

# Define line types for different model types
LSE_linestyle = '--'
direct_linestyle = '-'

# Define colors for LSE and direct models
colors = ['grey', 'red', 'blue', 'green', 'black']

# Initialize a dictionary to hold accuracies for each model type across trials
all_accuracies = {}

# Loop over each path and extract accuracies
for path in paths:
    with open(path, 'rb') as file:
        experimentInfo = pickle.load(file)
    # Extract data from experimentInfo
    _, _, accuracies = experimentInfo
    # Append accuracies to all_accuracies dictionary
    for model_type, acc in accuracies.items():
        if model_type not in all_accuracies:
            all_accuracies[model_type] = []
        all_accuracies[model_type].append(acc)

# Calculate mean and std deviation for each model type
mean_accuracies = {}
std_accuracies = {}

for model_type, acc_list in all_accuracies.items():
    acc_array = np.array(acc_list)
    mean_accuracies[model_type] = np.mean(acc_array, axis=0)
    std_accuracies[model_type] = np.std(acc_array, axis=0)

# Plotting the mean accuracies with error bars and caps
for model_type, mean_acc in mean_accuracies.items():
    std_acc = std_accuracies[model_type]
    
    if 'norm' in model_type:
        model_idx = 0
    elif 'dist' in model_type:
        model_idx = 1
    elif 'dot' in model_type:
        model_idx = 2
    elif 'unconnected' in model_type:
        model_idx = 3
    else:
        model_idx = 4

    color = colors[model_idx]

    if 'LSE' in model_type:
        linestyle = LSE_linestyle
    elif 'Direct' in model_type:
        linestyle = direct_linestyle

    epochs = range(len(mean_acc))

    if model_type == 'conscious_model':
        plt.errorbar(epochs, mean_acc, yerr=std_acc, label='conscious', color=color, linestyle=direct_linestyle, capsize=5)
    elif model_type =='unconnected_model':
        plt.errorbar(epochs, mean_acc[:, 0], yerr=std_acc[:, 0], label=(f'{model_type} (Direct)'), color=color, linestyle='-', capsize=5)
        plt.errorbar(epochs, mean_acc[:, 1], yerr=std_acc[:, 1], label=(f'{model_type} LSE'), color=color, linestyle='--', capsize=5)

plt.title('Validation Accuracy of InfoNCE Models')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
def best(acc):
    if type(acc[0]) is tuple:
        return max([x[0] for x in acc]), max([x[1] for x in acc])
    else:
        return max(acc)

In [ ]:
from collections import defaultdict

temperatures = [0.001, 0.01]

for temp in temperatures:
    paths = df.loc[(df['trial']==0) & (df['feature_dim'] == 4) & (df['temperature'] == temp), ['experimentInfo_path', 'l2_penalty']].reset_index(drop=True)

    penalties = []
    performances = defaultdict(list)


    for _, row in paths.iterrows():
        penalty = row['l2_penalty']
        penalties.append(penalty)
        p = row['experimentInfo_path']

        with open(p, 'rb') as file:
            experimentInfo = pickle.load(file)

        distribution, models, accuracies = experimentInfo

        for model_type, acc in accuracies.items():
            
            if model_type == 'conscious_model':
                performances[model_type].append(best(acc))
            else:
                direct, LSE = best(acc)
                performances[f'{model_type} (Direct)'].append(direct)
                performances[f'{model_type} (LSE)'].append(LSE)

    print(performances)

    # Convert numerical penalties to string labels to treat them as categorical
    penalty_labels = [str(p) for p in penalties]

    # Create a figure and axis for plotting
    fig, ax = plt.subplots()

    for name, perf in performances.items():
        # Generate a sequence of indices as x-coordinates for plotting
        x_coords = range(len(perf))
        # Plot using the generated x-coordinates and corresponding performance values
        ax.plot(x_coords, perf, label=name, marker='o')  # Adding 'marker' to denote each data point

        # Set the x-axis ticks to correspond to the penalty labels
        ax.set_xticks(x_coords)
        ax.set_xticklabels(penalty_labels, rotation=45)  # Rotate labels to avoid overlap

    ax.set_xlabel('L2 Penalty')
    ax.set_ylabel('Accuracy')
    plt.title(f'Performance Across Different L2 Penalties w/ temp={temp}')
    plt.legend()
    plt.tight_layout()  # Adjust layout to make room for the rotated x-axis labels
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict

# Define line types for different model types
LSE_linestyle = '--'
direct_linestyle = '-'

# Define colors for LSE and direct models
colors = ['grey', 'red', 'blue']


# Assuming 'df' is your DataFrame already loaded with data
temperatures = [0.001, 0.010]  # Example temperatures

# Setting up a 2x3 grid for subplots
fig, axs = plt.subplots(2, 3, figsize=(10, 10))  # Adjust the figure size as needed

# Flatten the axs array for easy indexing
axs = axs.flatten()
acc_dict = defaultdict(lambda: 0)

for t in range(10):
    best_dict = defaultdict(lambda: 0)
    for temp_index, temp in enumerate(temperatures):
        paths = df.loc[(df['trial'] == t) 
                       & (df['feature_dim'] == 8) 
                       & (df['temperature'] == temp), ['experimentInfo_path', 'l2_penalty']].reset_index(drop=True)

        penalties = []
        performances = defaultdict(list)
    
        for _, row in paths.iterrows():
            penalty = row['l2_penalty']
            if penalty >= 0.5:
                continue
            penalties.append(penalty)
            p = row['experimentInfo_path']

            with open(p, 'rb') as file:
                experimentInfo = pickle.load(file)

            distribution, models, accuracies = experimentInfo

            for model_type, acc in accuracies.items():
                if model_type == 'conscious_model':
                    performances[model_type].append(best(acc))
                else:
    #                 if model_type == 'norm_model':
    #                     continue
                    direct, LSE = best(acc)
                    performances[f'{model_type} (Direct)'].append(direct)
                    performances[f'{model_type} (LSE)'].append(LSE)

        penalty_labels = [str(p) for p in penalties]
        x_coords = range(len(penalty_labels))
        
        
        for i, (name, perf) in enumerate(performances.items()):
            best_dict[name] = max(best_dict[name], max(perf))
            
        

        # Plot on the current subplot
        ax = axs[temp_index]
        for i, (name, perf) in enumerate(performances.items()):
            if 'norm' in name:
                model_idx = 0
            elif 'dist' in name:
                model_idx = 1
            elif 'dot' in name:
                model_idx = 2
            else:
                continue

            color = colors[model_idx]

            if 'LSE' in name:
                linestyle = LSE_linestyle
            elif 'Direct' in name:
                linestyle = direct_linestyle

            # linestyle = model_linestyles[model_idx]

            ax.plot(x_coords, perf, label=name, marker='o', color=color, linestyle=linestyle)

        ax.set_xticks(x_coords)
        ax.set_xticklabels(penalty_labels, rotation=45)
        ax.set_title(f'Temperature: {temp}')
        ax.set_xlabel('L2 Penalty')
        ax.set_ylabel('Performance')
        ax.legend()
    
    print(best_dict)
    for k, v in best_dict.items():
        acc_dict[k] = acc_dict[k] + v
    # If there are any unused subplots (in case of less than 6 temperatures), hide them
    for i in range(len(temperatures), 6):
        axs[i].axis('off')

    plt.tight_layout()
    plt.show()

print(f'ACC DICT')
print(acc_dict)

In [ ]:


"""
Feature Embedding = 4
{'norm_model (Direct)': 8.334677419354838,
             'norm_model (LSE)': 7.056451612903226,
             'dist_model (Direct)': 9.179435483870966,
             'dist_model (LSE)': 9.039314516129032,
             'dot_model (Direct)': 1.9455645161290325,
             'dot_model (LSE)': 6.94657258064516,
             'unconnected_model (Direct)': 0.8306451612903226,
             'unconnected_model (LSE)': 7.601814516129032,
             'conscious_model': 9.194556451612902})
Feature Embedding - 8
{'norm_model (Direct)': 9.651209677419356,
             'norm_model (LSE)': 7.56149193548387,
             'dist_model (Direct)': 9.557459677419354,
             'dist_model (LSE)': 9.495967741935486,
             'dot_model (Direct)': 3.9627016129032255,
             'dot_model (LSE)': 7.410282258064517,
             'unconnected_model (Direct)': 0.779233870967742,
             'unconnected_model (LSE)': 9.089717741935484,
             'conscious_model': 9.703629032258064}
"""
acc_dict

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Set figure size
color_map = {'norm': 'orange', 'dist': 'skyblue', 'dot_': 'red', 'cons': 'gray', 'unco': 'green'}
plt.figure(figsize=(12, 8))  # Adjust the values to make it wider

# Create bar plot
bars = plt.bar(acc_dict.keys(), [x / 10 for x in acc_dict.values()], width=0.6)

# Function to darken a color
def darken_color(color, factor=0.7):
    # Convert color to RGB
    rgb = mcolors.to_rgb(color)
    # Darken the color by reducing brightness
    return tuple([x * factor for x in rgb])

# Color the bars and set edge color to a darker shade
for bar, key in zip(bars, acc_dict.keys()):
    color_code = key[:4]
    if color_code in color_map:
        main_color = color_map[color_code]
        bar.set_color(main_color)
        bar.set_edgecolor(darken_color(main_color))
        bar.set_linewidth(4)
    else:
        # Use a default color if the first 4 digits don't match any in the color map
        bar.set_color('gray')
        bar.set_alpha(0.5)  # Make the default color semi-transparent
        bar.set_edgecolor(darken_color('gray'))

# Add labels and title
plt.xlabel('InfoNCE Variants', fontsize=14)
plt.ylabel('Retrieval Accuracy', fontsize=14)
# plt.title('Accuracy of Differnt Variants of InfoNCE', fontsize=16)

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    plt.annotate('{:.3f}'.format(height),
                 xy=(bar.get_x() + bar.get_width() / 2, height),
                 xytext=(0, 3),
                 textcoords="offset points",
                 ha='center', va='bottom', fontsize=12)

# Rotate x-axis labels for better visibility
plt.xticks(rotation=45)

# Add grid
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Adjust spacing between subplots
plt.tight_layout()

# Display the plot
plt.show()


### Check embedding Gaussianity.

Holding fixed feature dimension 2, for the grid temperatures x l2_penalty, what percentage of the embeddings are Gaussian?

IMPORTANT: Want to highlight diff between Inner Product InfoNCE and L2 Distance InfoNCE

In [ ]:
import pingouin as pg 

# TODO: code here, plot confusion matrix
feature_dim = 2
temps = [0.001, 0.01, 0.1, 1.]
l2_penalties = [0.0, 0.001, 0.01, 0.1, 0.5]

gaussian_grid_dot = np.zeros((len(temps), len(l2_penalties)))
gaussian_grid_dist = np.zeros((len(temps), len(l2_penalties)))

# IMPORTANT, CHANGE NOISE, for 128_run it is 2.5 2.5 for others it is 1.5 1.5
for idx, temp in enumerate(temps):
    print('temp')
    for idy, l2_penalty in enumerate(l2_penalties):
        print(f'{temp}, {l2_penalty}')
        gaussian_counter = defaultdict(list)
        
        paths = df.loc[(df['feature_dim'] == feature_dim) 
                       & (df['temperature'] == temp)
                       & (df['l2_penalty'] == l2_penalty)
                       , ['experimentInfo_path', 'l2_penalty']].reset_index(drop=True)
        plotted = False
        for _, row in paths.iterrows():
            p = row['experimentInfo_path']

            with open(p, 'rb') as file:
                experimentInfo = pickle.load(file)

            distribution, models, accuracies = experimentInfo
            
            mu = distribution['mu']
            Sigma = distribution['Sigma']
            transform1 = distribution['transform1']
            transform2 = distribution['transform2']
            
            DIMS = (mu.shape[0], transform1.shape[0], transform2.shape[0])
            
            cum_dims = [sum(DIMS[:i+1]) for i in range(len(DIMS))]
            
            val_set = JointGaussianDataset(mu, 
                                           Sigma,
                                           transform1=transform1, 
                                           transform2=transform2, 
                                           pair=[0, 1], # unimportant
                                           dims=DIMS, 
                                           num_points=1000, 
                                           noise=(2.5, 2.5))
            
            points = torch.tensor(val_set.points)
            for model_type, model in models.items():
                
                PHI_B = model.encode(points[0:300, :cum_dims[0]], model.encoderA)
                # Convert PHI_B to a NumPy array for compatibility with statistical tests
                PHI_B_np = PHI_B.detach().numpy()
                if not plotted and (model_type == 'dot_model' or model_type == 'dist_model' or model_type == 'norm_model'):
                    plt.scatter(PHI_B_np[:, 0], PHI_B_np[:, 1], label=model_type)

                # Perform the Henze-Zirkler multivariate normality test
                hz_test = pg.multivariate_normality(PHI_B_np, alpha=.05)
                # print(f"Henze-Zirkler Test on PHI_B_dist: {hz_test}")
                if model_type == 'dist_model':
                    gaussian_grid_dist[idx, idy] += hz_test.pval
                elif model_type == 'dot_model':
                    gaussian_grid_dot[idx, idy] += hz_test.pval
                
            plotted = True
        plt.title(f'InfoNCE Embedding Distribution')
        plt.xlabel('x_1')
        plt.ylabel('x_2')
        plt.legend()
        plt.show()

print(len(paths))
gaussian_grid_dot /= len(paths)
gaussian_grid_dist /= len(paths)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print(gaussian_grid_dist)

plt.figure(figsize=(10, 7))
sns.heatmap(gaussian_grid_dist[:4,:4], annot=True, cmap='Blues', xticklabels=l2_penalties[:4], yticklabels=temps)

# Adding labels for axes
plt.xlabel('l2 penalty')
plt.ylabel('temperature')

plt.title('Is L2 Dist InfoNCE Gaussian? (Average p-val)')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
gaussian_grid_dot = np.nan_to_num(gaussian_grid_dot)
print(gaussian_grid_dot)

plt.figure(figsize=(10, 7))
sns.heatmap(gaussian_grid_dot[:4,:4], annot=True, cmap='Blues', xticklabels=l2_penalties[:4], yticklabels=temps[:4])

# Adding labels for axes
plt.xlabel('l2 penalty')
plt.ylabel('temperature')

plt.title('Is Inner Product InfoNCE Gaussian? (Average p-val)')
plt.show()
